### Part Raúl - Elections 1931, 1936, 1939, 1945 & 1950

In [18]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from io import StringIO

# Initialize the WebDriver
driver = webdriver.Chrome()

# Base URL
enlace_base = 'https://infogob.jne.gob.pe/Eleccion'

# List to store tables
elections_1931_1950 = []

# We are selecting the last 5 elections
for k in range(21, 26):
    driver.get(enlace_base)
    time.sleep(2)

    # Select "Tipo de Proceso"
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div').click()
    time.sleep(2)
    # Combine moving down and selecting the option into a single ActionChain
    ActionChains(driver).send_keys(Keys.ARROW_DOWN).send_keys(Keys.ENTER).perform()
    time.sleep(2)  # Add a minimal delay if needed for the page to react

    # Select "Elección"
    dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    dropdown.click()
    time.sleep(1)  # Short delay to ensure the dropdown is open

    # Move down in the drop-down list k times
    for _ in range(k):
        dropdown.send_keys(Keys.ARROW_DOWN)
        time.sleep(0.5)  # Adjust delay to ensure the dropdown responds

    # Select the currently highlighted option
    dropdown.send_keys(Keys.ENTER)
    time.sleep(2)  # Adjust delay to ensure the selection is made

    # Extract the selected election name
    selected_election_name = dropdown.text

    # View data
    driver.find_element(By.XPATH, '//*[@id="btnVerDatos"]').click()
    time.sleep(10)

    # View results table
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a').click()
    time.sleep(2)

    # Extract table data
    table_html = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
    table_data = pd.read_html(StringIO(table_html))[0]

    # Add the election name as a new column to the table
    table_data['Elección'] = selected_election_name

    # Keep only the desired columns
    table_data = table_data[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    elections_1931_1950.append(table_data)

    time.sleep(2)            

    print(f"Data for election number {k}:\n", table_data)
    print("\n" + "=" * 50 + "\n")

driver.quit()

# Concatenate all tables into a single DataFrame
table_1931_1950 = pd.concat(elections_1931_1950, ignore_index=True)

# Save the final table to an Excel file
table_1931_1950.to_excel('Elecciones_Data.xlsx', index=False)

Data for election number 21:
             Elección ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1950   PARTIDO RESTAURADOR       550779


Data for election number 22:
             Elección        ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1945  FRENTE DEMOCRATICO NACIONAL       305590
1  PRESIDENCIAL 1945         UNION REVOLUCIONARIA       150720


Data for election number 23:
             Elección   ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1939  CONCENTRACION NACIONAL       262971
1  PRESIDENCIAL 1939       FRENTE PATRIOTICO        76222


Data for election number 24:
             Elección     ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1936  PARTIDO SOCIAL DEMOCRATA        77042
1  PRESIDENCIAL 1936      UNION REVOLUCIONARIA        57838
2  PRESIDENCIAL 1936           FRENTE NACIONAL        46773
3  PRESIDENCIAL 1936       PARTIDO REPUBLICANO        30803


Data for election number 25:
             Elección       ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PR

In [20]:
## Finally, we can see the required table
table_1931_1950

,Elección,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 1950,PARTIDO RESTAURADOR,550779
1,PRESIDENCIAL 1945,FRENTE DEMOCRATICO NACIONAL,305590
2,PRESIDENCIAL 1945,UNION REVOLUCIONARIA,150720
3,PRESIDENCIAL 1939,CONCENTRACION NACIONAL,262971
4,PRESIDENCIAL 1939,FRENTE PATRIOTICO,76222
5,PRESIDENCIAL 1936,PARTIDO SOCIAL DEMOCRATA,77042
6,PRESIDENCIAL 1936,UNION REVOLUCIONARIA,57838
7,PRESIDENCIAL 1936,FRENTE NACIONAL,46773
8,PRESIDENCIAL 1936,PARTIDO REPUBLICANO,30803
9,PRESIDENCIAL 1931,UNION REVOLUCIONARIA,152149


### Part Luis Felipe - Elections 1990 (1st round), 1990 (2nd round), 1995, 2000 (1st round) & 2000 (2nd round)

In [31]:
# Initialize the WebDriver
driver = webdriver.Chrome()

# Base URL
enlace_base = 'https://infogob.jne.gob.pe/Eleccion'

# List to store tables
elections_1990_2000 = []

# We are selecting the last 5 elections
for k in range(11, 15):
    driver.get(enlace_base)
    time.sleep(2)

    # Select "Tipo de Proceso"
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div').click()
    time.sleep(2)
    # Combine moving down and selecting the option into a single ActionChain
    ActionChains(driver).send_keys(Keys.ARROW_DOWN).send_keys(Keys.ENTER).perform()
    time.sleep(2)  # Add a minimal delay if needed for the page to react

    # Select "Elección"
    dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    dropdown.click()
    time.sleep(1)  # Short delay to ensure the dropdown is open

    # Move down in the drop-down list k times
    for _ in range(k):
        dropdown.send_keys(Keys.ARROW_DOWN)
        time.sleep(0.5)  # Adjust delay to ensure the dropdown responds

    # Select the currently highlighted option
    dropdown.send_keys(Keys.ENTER)
    time.sleep(2)  # Adjust delay to ensure the selection is made

    # Extract the selected election name
    selected_election_name = dropdown.text

    # View data
    driver.find_element(By.XPATH, '//*[@id="btnVerDatos"]').click()
    time.sleep(10)

    # View results table
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a').click()
    time.sleep(2)

    # Extract table data
    table_html = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
    table_data = pd.read_html(StringIO(table_html))[0]

    # Add the election name as a new column to the table
    table_data['Elección'] = selected_election_name

    # Keep only the desired columns
    table_data = table_data[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    elections_1990_2000.append(table_data)

    time.sleep(2)            

    print(f"Data for election number {k}:\n", table_data)
    print("\n" + "=" * 50 + "\n")

driver.quit()

# Concatenate all tables into a single DataFrame
table_1990_2000 = pd.concat(elections_1990_2000, ignore_index=True)

# Save the final table to an Excel file
table_1990_2000.to_excel('Elecciones_Data.xlsx', index=False)

Data for election number 11:
                          Elección        ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 2000 - 2DA VUELTA  ALIANZA ELECTORAL PERU 2000      6041774
1  PRESIDENCIAL 2000 - 2DA VUELTA                 PERU POSIBLE      2086208
2  PRESIDENCIAL 2000 - 2DA VUELTA              VOTOS EN BLANCO       140773
3  PRESIDENCIAL 2000 - 2DA VUELTA                  VOTOS NULOS      3531730


Data for election number 12:
             Elección                       ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 2000                 ALIANZA ELECTORAL PERU 2000      5528394
1  PRESIDENCIAL 2000                                PERU POSIBLE      4460812
2  PRESIDENCIAL 2000         MOVIMIENTO INDEPENDIENTE SOMOS PERU       333049
3  PRESIDENCIAL 2000          AGRUPACION INDEPENDIENTE AVANCEMOS       246781
4  PRESIDENCIAL 2000                        SOLIDARIDAD NACIONAL       199813
5  PRESIDENCIAL 2000                     PARTIDO APRISTA PERUANO       152519
6  PRESIDENC

In [32]:
## Finally, we can see the required table
table_1990_2000

,Elección,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 2000 - 2DA VUELTA,ALIANZA ELECTORAL PERU 2000,6041774
1,PRESIDENCIAL 2000 - 2DA VUELTA,PERU POSIBLE,2086208
2,PRESIDENCIAL 2000 - 2DA VUELTA,VOTOS EN BLANCO,140773
3,PRESIDENCIAL 2000 - 2DA VUELTA,VOTOS NULOS,3531730
4,PRESIDENCIAL 2000,ALIANZA ELECTORAL PERU 2000,5528394
5,PRESIDENCIAL 2000,PERU POSIBLE,4460812
6,PRESIDENCIAL 2000,MOVIMIENTO INDEPENDIENTE SOMOS PERU,333049
7,PRESIDENCIAL 2000,AGRUPACION INDEPENDIENTE AVANCEMOS,246781
8,PRESIDENCIAL 2000,SOLIDARIDAD NACIONAL,199813
9,PRESIDENCIAL 2000,PARTIDO APRISTA PERUANO,152519
